In [13]:
import pickle
import numpy as np
import sympy
import pandas as pd
import sys
import time
import cartpole_error_monitoring




In [14]:
project_name = "cartpole_error_monitoring"
# 読み込みたいファイル名のリスト
filenames = ['./log/'+project_name+'_uopt.log', './log/'+project_name+'_x.log', './log/'+project_name+'_t.log']

# 各ファイルを読み込み、その結果をリストに保存します
df_list = [pd.read_csv(filename, delim_whitespace=True, header=None) for filename in filenames]

# 各データフレームをnumpyの行列に変換し、それをリストに保存します
u_matrix, x_matrix, t = [df.values for df in df_list]


In [15]:
import os, json
from math import sqrt
import cgmres.cartpole_error_monitoring
import cgmres.common


from matplotlib import pyplot as plt

config_name = 'config_20230722.json'

with open('./config/'+config_name, 'r') as f:
    config = json.load(f)

ocp = cgmres.cartpole_error_monitoring.OCP()

horizon = cgmres.common.Horizon(config["Tf"]) # fixed length

settings = cgmres.common.SolverSettings()
settings.sampling_time = config["sampling_time"]
settings.zeta = config["zeta"]
settings.finite_difference_epsilon = config["finite_difference_epsilon"]
settings.max_iter = config["max_iterations"]
settings.opterr_tol = config["tolerance"]
settings.verbose_level = 1 # print opt error

t0 = config["initial_time"]
x0 = np.zeros(ocp.nx)

# Initialize solution using zero horizon OCP solution
initializer = cgmres.cartpole_error_monitoring.ZeroHorizonOCPSolver(ocp, settings)
# uc0 = np.array([0.01])
# initializer.set_uc(uc0)
# initializer.solve(t0, x0)

# simple simulation with forward Euler 
simulation_time = config["simulation_length"]
sampling_time = settings.sampling_time
t = t0
x = x0.copy()
for k in range(int(simulation_time/sampling_time)):
    u = mpc.uopt[0]
    dx = ocp.eval_f(t, x, u)
    x1 = x + sampling_time * dx
    mpc.update(t, x)
    x = x1
    t = t + sampling_time
    print('t: ', t, ', x: ', x)

print("\n======================= MPC used in this simulation: =======================")
print(mpc)



In [ ]:
# Create and configure the AutoGenU object
# auto_gen_u = cartpole_error_monitoring.create_autogenu_object(config)
# cartpole_error_monitoring.set_parameters(auto_gen_u, config)


# x = sympy.Matrix(sympy.symbols(f'x0:{config["nx"]}'))  # Initial state as variables
# u = sympy.Matrix([sympy.symbols(f'u{i}_{j}') for i in range(config["nu"]) for j in range(config["N"])]).reshape(config["nu"], config["N"])
# r = sympy.symbols('r[0:%d]' %(config["nu"]))
# q_terminal = sympy.symbols('q_terminal[0:%d]' %(config["nx"]))
# q = sympy.symbols('q[0:%d]' %(config["nx"]))
# x_ref = sympy.symbols('x_ref[0:%d]' %(config["nx"]))


# sub_param = auto_gen_u.make_subs_dicts()

# # Create a list to store the L2 norms and F values
# l2_norms = []
# F_param_substituted = []

# simulation_steps = int(config["simulation_length"]/config["sampling_time"])
# print("simulation step:", simulation_steps)
# step = 10
# start_time = time.time()
# # for k in range(int(config["simulation_length"]/config["sampling_time"])):
# for k in range(step):
#     print(k)
#     # Create the substitutions for the state and input at the current time step
#     subs_x = [(x[l], x_matrix[k,l]) for l in range(config["nx"])]
#     subs_u = [(u[l], u_matrix[k,l]) for l in range(config["N"])]

#     # Create a list to store the F values for the current time step
#     F_param_substituted_step = []

#     # Loop over the control inputs
#     norm = 0
#     for i in range(config["N"]):
#         for j in range(config["nu"]):
#             # Calculate the value of F
#             F_val = F[i,j].subs(sub_param).subs(subs_x).subs(subs_u)

#             F_param = F[i,j].subs(sub_param)
#             F_param_substituted_step.append(F_param)
            
#             # Add to the squared L2 norm
#             norm += F_val**2

    
#     # if k%50 == 0:
#     #     print(k)
#     # Append the square root of the norm (i.e., the L2 norm) to the list
#     l2_norms.append(sqrt(norm))
#     end_time = time.time()
#     print("computation time in %d step: %f" % (k, end_time - start_time))


#     # Append the F values for the current step to the list
#     # F_param_substituted.append(F_param_substituted_step)

# simulation_steps = int(config["simulation_length"]/config["sampling_time"])
# t_series = [item for sublist in t[0:step] for item in sublist]
# result = np.log10(l2_norms  )
# plt.plot(t_series,result)
# plt.ylabel(r'$\log_{10}\Vert F \Vert_2$')
# plt.xlabel('Time (s)')